# LGBM - Training

Sources
1. [LGBM & Deberta Explained by ZULQARNAIN ALI](https://www.kaggle.com/code/zulqarnainalipk/lgbm-deberta-explained)

## Setup

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
import lightgbm as lgb
import pickle as pkl
import wandb
from lightgbm import log_evaluation, early_stopping
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold

In [2]:
os.chdir("../../")

In [3]:
from lib.config import config
from lib.paths import Paths
from lib.utils.utils import seed_everything
from lib.criterion.metrics import log_metrics
from lib.model.utils import quadratic_weighted_kappa, qwk_obj, get_score

In [4]:
seed_everything()

### Setting Up WandB

In [5]:
WANDB_PROJECT = "Kaggle_ASE_2.0"
WANDB_NAME = f"LGBM-ASE-7"

In [6]:
wandb.login(key=os.environ.get('WANDB_API_KEY'))
wandb.init(
    project=WANDB_PROJECT,
    name=WANDB_NAME,
    save_code=True,
    job_type="train",
    config=config,
)
config = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shakleenishfar (laplacesdemon43). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ishfar/.netrc


## Model Training

### Dataset Creation

In [7]:
df = pd.read_csv(Paths.FEATURE_ENGG_CSV_PATH)
df.shape

(17307, 220)

In [8]:
ids = df.loc[:, "essay_id"]
X, y = df.drop(columns=["score", "essay_id"]), df.loc[:, "score"]
del df

X.shape, y.shape

((17307, 218), (17307,))

### Callbacks

In [9]:
callbacks = [
    log_evaluation(period=25),
    early_stopping(stopping_rounds=75, first_metric_only=True),
]

### Constants

In [10]:
a = config.lgbm_a
b = config.lgbm_b

### Training Loop

In [11]:
def train_loop(X_train, y_train, X_valid, y_valid):
    y_train = y_train.astype(np.float32) - a
    y_valid = y_valid.astype(np.float32) - a

    model = lgb.LGBMRegressor(
        objective=qwk_obj,
        metrics="None",
        learning_rate=0.01,
        max_depth=5,
        num_leaves=10,
        colsample_bytree=0.3,
        reg_alpha=0.7,
        reg_lambda=0.1,
        n_estimators=700,
        random_state=config.random_seed,
        extra_trees=True,
        class_weight="balanced",
        n_jobs=6,
        verbosity=0,
    )

    # Train model
    predictor = model.fit(
        X_train,
        y_train,
        eval_names=["train", "valid"],
        eval_set=[(X_train, y_train), (X_valid, y_valid)],
        eval_metric=quadratic_weighted_kappa,
        callbacks=callbacks,
    )

    return model, predictor

In [12]:
def validate_model(idx, X_valid, y_valid, id_valid, predictor):
    y_valid_int = y_valid.astype(np.int64).to_numpy()
    preds = predictor.predict(X_valid) + a
    preds = preds.clip(1, 6).round().astype(np.int64)

    oof = pd.DataFrame(
        {
            "essay_id": id_valid,
            "score": y_valid_int - 1,
            "pred_score": preds - 1,
        }
    )
    log_metrics(oof, f"Fold {idx}")

    return oof

### Fold Loop

In [13]:
skf = StratifiedKFold(
    n_splits=config.lgbm_n_folds,
    random_state=config.random_seed,
    shuffle=True,
)

In [14]:
oof_df = pd.DataFrame()

for idx, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, y_train = X.loc[train_idx], y.loc[train_idx]
    X_valid, y_valid = X.loc[valid_idx], y.loc[valid_idx]
    id_valid = ids.loc[valid_idx].to_numpy()

    model, predictor = train_loop(X_train, y_train, X_valid, y_valid)

    with open(f"output/LGBM/{idx}.pkl", "wb") as file:
        pkl.dump({"predictor": predictor, "model": model}, file)

    oof = validate_model(idx, X_valid, y_valid, id_valid, predictor)
    oof_df = pd.concat([oof_df, oof])

[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.446787	valid's QWK: 0.469265
[50]	train's QWK: 0.73307	valid's QWK: 0.751725
[75]	train's QWK: 0.798825	valid's QWK: 0.8077
[100]	train's QWK: 0.837112	valid's QWK: 0.844476
[125]	train's QWK: 0.859903	valid's QWK: 0.863581
[150]	train's QWK: 0.868852	valid's QWK: 0.876823
[175]	train's QWK: 0.873272	valid's QWK: 0.881612
[200]	train's QWK: 0.875598	valid's QWK: 0.886958
[225]	train's QWK: 0.877755	valid's QWK: 0.890796
[250]	train's QWK: 0.880503	valid's QWK: 0.891972
[275]	train's QWK: 0.8821	valid's QWK: 0.892983
[300]	train's QWK: 0.882991	valid's QWK: 0.894687
[325]	train's QWK: 0.883816	valid's QWK: 0.89671
[350]	train's QWK: 0.884678	valid's QWK: 0.897519
[375]	train's QWK: 0.885049	valid's QWK: 0.897074
[400]	train's QWK: 0.885628	valid's QWK: 0.896483
Early stopping, best iteration is:
[331]	train's QWK: 0.884172	valid's QWK: 0.898293
Evaluat

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.446482	valid's QWK: 0.445172
[50]	train's QWK: 0.741632	valid's QWK: 0.712405
[75]	train's QWK: 0.797625	valid's QWK: 0.770586
[100]	train's QWK: 0.834037	valid's QWK: 0.813425
[125]	train's QWK: 0.860438	valid's QWK: 0.844492
[150]	train's QWK: 0.869439	valid's QWK: 0.850799
[175]	train's QWK: 0.874118	valid's QWK: 0.855183
[200]	train's QWK: 0.876973	valid's QWK: 0.859066
[225]	train's QWK: 0.880068	valid's QWK: 0.860806
[250]	train's QWK: 0.881694	valid's QWK: 0.865068
[275]	train's QWK: 0.883579	valid's QWK: 0.866535
[300]	train's QWK: 0.883898	valid's QWK: 0.864379
[325]	train's QWK: 0.884593	valid's QWK: 0.863426
Early stopping, best iteration is:
[261]	train's QWK: 0.882608	valid's QWK: 0.867899
Evaluated only: QWK
[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's Q

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.431555	valid's QWK: 0.444834
[50]	train's QWK: 0.739566	valid's QWK: 0.748673
[75]	train's QWK: 0.800859	valid's QWK: 0.802043
[100]	train's QWK: 0.832937	valid's QWK: 0.831732
[125]	train's QWK: 0.858483	valid's QWK: 0.863088
[150]	train's QWK: 0.868251	valid's QWK: 0.872035
[175]	train's QWK: 0.872118	valid's QWK: 0.87393
[200]	train's QWK: 0.875428	valid's QWK: 0.875947
[225]	train's QWK: 0.878543	valid's QWK: 0.87555
[250]	train's QWK: 0.880822	valid's QWK: 0.87819
[275]	train's QWK: 0.882891	valid's QWK: 0.879608
[300]	train's QWK: 0.883766	valid's QWK: 0.8818
[325]	train's QWK: 0.8845	valid's QWK: 0.882006
[350]	train's QWK: 0.885188	valid's QWK: 0.881433
[375]	train's QWK: 0.88576	valid's QWK: 0.88043
Early stopping, best iteration is:
[318]	train's QWK: 0.884245	valid's QWK: 0.882596
Evaluated only: QWK
[LightGBM] [Info] Using self-defined obj

/media/ishfar/New Volume/Studies/Projects/Kaggle-Automated-Essay-Scoring/lib/criterion/metrics.py:34: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


[LightGBM] [Info] Using self-defined objective function
Training until validation scores don't improve for 75 rounds
[25]	train's QWK: 0.452197	valid's QWK: 0.479338
[50]	train's QWK: 0.737333	valid's QWK: 0.729829
[75]	train's QWK: 0.801673	valid's QWK: 0.78583
[100]	train's QWK: 0.836949	valid's QWK: 0.825158
[125]	train's QWK: 0.85924	valid's QWK: 0.846688
[150]	train's QWK: 0.869229	valid's QWK: 0.858699
[175]	train's QWK: 0.873174	valid's QWK: 0.870372
[200]	train's QWK: 0.876604	valid's QWK: 0.866876
[225]	train's QWK: 0.878609	valid's QWK: 0.870686
[250]	train's QWK: 0.880829	valid's QWK: 0.870806
[275]	train's QWK: 0.882262	valid's QWK: 0.874716
[300]	train's QWK: 0.883378	valid's QWK: 0.876717
[325]	train's QWK: 0.883991	valid's QWK: 0.879193
[350]	train's QWK: 0.884582	valid's QWK: 0.880472
[375]	train's QWK: 0.885157	valid's QWK: 0.88224
[400]	train's QWK: 0.885642	valid's QWK: 0.88224
[425]	train's QWK: 0.886012	valid's QWK: 0.883994
[450]	train's QWK: 0.88636	valid's QWK: 

In [15]:
score = get_score(oof_df["score"], oof_df["pred_score"])
print(f'Overall Score: {score:<.4f}')
wandb.log({"CV/qwk_score": score})

Overall Score: 0.8862


In [16]:
wandb.log({"oof_table": wandb.Table(dataframe=oof_df)})

In [17]:
log_metrics(
    pd.DataFrame(
        {
            "score": oof_df.score.to_numpy(),
            "pred_score": oof_df.pred_score.to_numpy(),
        }
    ),
    "Overall",
)

In [18]:
wandb.finish()

CV/qwk_score,▁
CV/qwk_score,0.88615
